In [1]:
# @title
import yfinance as yf
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from datetime import datetime
from dateutil.relativedelta import relativedelta

In [2]:
#@title Seleccione Acción y Medias Móviles {run:'auto'}
ticker_symbol = "AAPL" #@param ['AAPL','ABT','ADBE','AMD','AMZN','ASML','AXP','AZN','BABA','BAC','BHP','BIP','BRK-B','CAT','COST','CSCO','CVX','GOOGL','GSK','JPM','KO','KO','LLY','LLYVK','LULU','LVMUY','MA','MA','MCD','MELI','META','MRK','MSFT','NFLX','NKE','NOW','NVDA','ORCL','PEP','PFE','PG','QCOM','QSR','RIO','SAP','SHEL','SNY','SPY','TM','TMO','TMUS','TOT','TSM','TXN','UBER','UNH','V','VZ','WM','XOM']
#warren = "AAPL" #@param  ["AAPL","AXP","BAC","KO","CVX","OXY","MCO","KHC","CB","DVA","VRSN","KR","V","SIRI","MA","AMZN","STZ","COF","UNH","AON","DPZ","ALLY","POOL","LLYVK","NUE","LEN","LPX","LLYVA","CHTR","DHI","LAMR","ALLE","NVR","JEF","DEO","LILA","LILAK","BATRK","HEI.A","LEN.B","FWONK"]

#ticker_symbol = "BTC-USD" #@param ["BTC-USD", "ETH-USD", "USDT-USD", "XRP-USD", "LTC-USD", "ADA-USD", "DOT-USD", "BCH-USD", "XLM-USD", "LINK-USD"]

end_date   =  datetime.today().strftime("%Y-%m-%d")
start_date = (datetime.today() - relativedelta(years=1)).strftime("%Y-%m-%d")

lookback_days = 180  # ventana para detectar el último swing
ratios = [0.236, 0.382, 0.5, 0.618, 0.786]  # retrocesos estándar


In [3]:
# @title

# -------------------------------
# 1) Descarga de datos
# -------------------------------
data = yf.download(ticker_symbol, start=start_date, end=end_date, auto_adjust=True)

# Normaliza columnas
if isinstance(data.columns, pd.MultiIndex):
    data.columns = data.columns.get_level_values(0)
data.columns = data.columns.str.lower()

if data.empty:
    raise ValueError("No se obtuvieron datos. Revisa ticker/fechas.")

df = data[['open','high','low','close', 'volume']].copy()

# -------------------------------
# 2) Detección automática del último swing
# -------------------------------
lookback_days = min(lookback_days, len(df))
window_df = df.tail(lookback_days)

low_price  = window_df['low'].min()
high_price = window_df['high'].max()
low_idx    = window_df['low'].idxmin()
high_idx   = window_df['high'].idxmax()

is_up_move = (low_idx < high_idx)  # True: tramo alcista (low->high); False: bajista (high->low)

# -------------------------------
# 3) Niveles de Fibonacci
# -------------------------------
levels = {}
if is_up_move:
    rng = high_price - low_price
    for r in ratios:
        levels[f"{int(r*100)}%"] = high_price - rng * r
    swing_text = f"Swing ALZA: Low={low_price:.2f} ({low_idx.date()}) → High={high_price:.2f} ({high_idx.date()})"
else:
    rng = high_price - low_price
    for r in ratios:
        levels[f"{int(r*100)}%"] = low_price + rng * r
    swing_text = f"Swing BAJA: High={high_price:.2f} ({high_idx.date()}) → Low={low_price:.2f} ({low_idx.date()})"

# Ordena niveles de menor a mayor (para usar bandas fácilmente)
levels_series = pd.Series(levels).sort_values()

# Asegura que existen las claves que usaremos para las bandas
lvl_38 = levels_series.get("38%")
lvl_50 = levels_series.get("50%")
lvl_61 = levels_series.get("61%")
lvl_78 = levels_series.get("78%")

print(swing_text)
print("\nNiveles de Fibonacci (retroceso):")
print(levels_series.to_frame("Precio"))

# -------------------------------
# 4) Gráfico + ZONAS DE COLOR
# -------------------------------
fig = go.Figure(data=[go.Candlestick(
    x=df.index, open=df['open'], high=df['high'], low=df['low'], close=df['close'],
    name=ticker_symbol
)])

# Rango vertical útil para cubrir las bandas
y_min = float(df['low'].min())
y_max = float(df['high'].max())

# Grafico: Volumen
fig1 = go.Figure(data=[go.Bar(
    x=df.index, y=df['volume'], 
    name=ticker_symbol
)])

# --- Bandas tipo semáforo ---
# 🔴 Rojo: por debajo de 61.8%
if lvl_61 is not None:
    fig.add_shape(
        type="rect",
        x0=df.index.min(), x1=df.index.max(),
        y0=y_min, y1=lvl_61,
        fillcolor="red", opacity=0.12, line_width=0, layer="below"
    )

# 🟡 Amarillo: entre 61.8% y 78.6%
if (lvl_61 is not None) and (lvl_78 is not None):
    y0, y1 = sorted([lvl_61, lvl_78])
    fig.add_shape(
        type="rect",
        x0=df.index.min(), x1=df.index.max(),
        y0=y0, y1=y1,
        fillcolor="yellow", opacity=0.12, line_width=0, layer="below"
    )

# 🟢 Verde: por arriba de 78.6%
if lvl_78 is not None:
    y0, y1 = sorted([lvl_78, y_max])
    fig.add_shape(
        type="rect",
        x0=df.index.min(), x1=df.index.max(),
        y0=y0, y1=y1,
        fillcolor="green", opacity=0.10, line_width=0, layer="below"
    )

# Líneas de Fibonacci
for name, price in levels_series.items():
    fig.add_hline(y=float(price), line_dash="dot", opacity=0.6,
                  annotation_text=name, annotation_position="right")

# Swing bounds
fig.add_hline(y=low_price,  line_dash="dash", opacity=0.3, annotation_text="Swing Low")
fig.add_hline(y=high_price, line_dash="dash", opacity=0.3, annotation_text="Swing High")

# Trazas dummy para leyenda de colores
fig.add_trace(go.Scatter(x=[None], y=[None], mode='lines',
                         line=dict(color='red'), name='Bajo 61.8% (zona de riesgo)'))
fig.add_trace(go.Scatter(x=[None], y=[None], mode='lines',
                         line=dict(color='yellow'), name='61.8%–78.6% (zona de decisión)'))
fig.add_trace(go.Scatter(x=[None], y=[None], mode='lines',
                         line=dict(color='green'), name='Sobre 78.6% (alcista)'))

fig.update_layout(
    title=f"{ticker_symbol} · Retrocesos de Fibonacci (con zonas de color)<br><sup>{swing_text}</sup>",
    xaxis_title="Fecha",
    yaxis_title="Precio",
    xaxis_rangeslider_visible=True,
    height=700,
    legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1)
)

fig1.update_layout(
    title=f"{ticker_symbol} · Volumen de Acciones <br>",
    xaxis_title="Fecha",
    yaxis_title="Volumen de Acciones",
    xaxis_rangeslider_visible=False,
    height=300,
    legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1)
)

fig.show()
fig1.show()

[*********************100%***********************]  1 of 1 completed


Swing BAJA: High=259.18 (2024-12-26) → Low=168.80 (2025-04-08)

Niveles de Fibonacci (retroceso):
         Precio
23%  190.127248
38%  203.323178
50%  213.988383
61%  224.653587
78%  239.837945


📍 Situación actual

*	El swing bajista fue 259 → 168 USD.
*	Hoy AAPL está rebotando fuerte y ya se metió en la franja amarilla (entre 61.8% y 78.6%), es decir: entre 225 y 240 USD aprox.


🍐🍎 Explicación sencilla (peras y manzanas)

* Imagina que AAPL se cayó de un árbol:
* Se subió hasta 259 (la rama más alta).
* Se resbaló y cayó hasta 168 (el suelo).
* Ahora se volvió a trepar otra vez por el tronco.
* En ese camino de regreso:
  * Pasó por la rama de 50% (214) → ya subió la mitad del tramo.
  * Ahora está entre las ramas 61.8% y 78.6% (225–240).
  * Estas ramas son las más gruesas y difíciles de superar: aquí los vendedores suelen decir “ya recuperaste demasiado, vendo mis acciones”.


📈 Expectativas

1.	Escenario positivo (rompe hacia arriba):
*	Si AAPL sube con fuerza y supera 240 USD (78.6%), la caída de 259→168 se invalida.
*	Entonces, lo más probable es que vaya directo a probar la rama más alta (259 USD).
2.	Escenario de debilidad (rechazo):
*	Si AAPL no puede sostenerse arriba de 225–240, puede retroceder.
*	Los soportes probables son: 214 (50%) o 203 (38%).
3.	Zona actual = zona de decisión.
*	No es momento de confiar ciegamente: el precio está en examen, el mercado decidirá si aprueba con rumbo a máximos o se regresa a estudiar a niveles más bajos.


🚦 Lectura tipo semáforo

*	🟢 Arriba de 240 USD (78.6%) → Alcista, alta probabilidad de volver a 259.
*	🟡 Entre 225 y 240 USD (61.8–78.6%) → Zona crítica, indecisión; puede romper hacia arriba o girar hacia abajo.
*	🔴 Debajo de 225 USD (61.8%) → Debilidad, riesgo de retroceder a 214 o 203.







📋 Plan de Trading AAPL – Basado en Retrocesos de Fibonacci (Swing 259 → 168)

|Estrategia|	Condición de Entrada|	Stop Loss|	Take Profit (Objetivos)	|Interpretación|
|---|---|---|---|---|
|🟢 Alcista	| Comprar si rompe 240 USD (arriba de 78.6%)|	Debajo de 224 USD (61.8%)|	🎯 259 (máximo previo)🎯 270–275 (extensión 127.2%)	|Si rompe con volumen invalida el swing bajista y busca nuevos máximos|
|🔴 Bajista	|Vender si falla entre 225–240 USD (rechazo zona amarilla)|	Arriba de 242–244 USD	|🎯 214 (50%)🎯 203 (38.2%)	Zona 61.8–78.6% |funciona como techo; el precio podría retroceder|
|🟡 Neutro	|No operar dentro de 225–240 USD|	–	|–	|Zona de indecisión; esperar confirmación fuera de la franja|



✅ Checklist para el trader

*	Confirmar dirección (¿rompe arriba de 240 o falla en 225–240?).
*	Verificar volumen y cierre diario.
*	Definir tamaño de posición según tolerancia al riesgo.
*	Respetar Stop Loss sin excepciones.
*	Tomar parciales en el primer objetivo y dejar correr el resto.

